In [7]:
import os

import pandas as pd
import grequests

def format_data_for_api(row_num, row_data):
    #'city': 'Los Angeles',

    data = {
        'street': row_data.Address.strip(),
        'state': 'CA',
        'zip': row_data.Zip,
        'format': 'json',
        'benchmark': 'Public_AR_Current',
        'row_num': row_num
    }
    return data

def format_data_from_api(resp):
    j_data = resp.json()
    addy_data = j_data['result']['addressMatches'][0]
    in_addy_data = j_data['result']['input']['address']
    out_dict = {
        'in_street': in_addy_data['street'],
        'in_zip': in_addy_data['zip'],
        'Address Start': addy_data['addressComponents']['fromAddress'],
        'Address End': addy_data['addressComponents']['toAddress'],
        'Street Direction': addy_data['addressComponents']['preDirection'],
        'Street Name': addy_data['addressComponents']['streetName'],
        'Street Suffix': addy_data['addressComponents']['suffixType'],
        'Zip Code': addy_data['addressComponents']['zip'],
        'Latitude/Longitude': (addy_data['coordinates']['y'], addy_data['coordinates']['x'])
    }
    return out_dict

In [ ]:
# Load Ellis dataset
DATA_DIR = '/Users/mcmenamin/GitHub/la_mayors_office/data'
DATA_FILE = 'To Mayor - Ellis 7-16-2007 - 7-31-2017 Ran 8-18-2017.xlsx'

OUT_FILE = 'ellis_geocoded.csv'

df_ellis = pd.read_excel(os.path.join(DATA_DIR, DATA_FILE))
df_ellis.Address = df_ellis.Address.str.strip()
df_ellis.Zip = df_ellis.Zip.astype(str)

# Set up async api calls

API_URL = "https://geocoding.geo.census.gov/geocoder/{returntype}/{searchtype}"

query_params = {
    'returntype': 'locations',
    'searchtype': 'address'
}

reqs = [grequests.get(API_URL.format(**query_params), params=format_data_for_api(row_num, row_data))
        for row_num, row_data in df_ellis.iterrows()]
resps = grequests.map(reqs)

In [ ]:
df_from_api = pd.DataFrame([format_data_from_api(r) for r in resps])

df_full = (
    df_from_api.merge(df_ellis, left_on=['in_street', 'in_zip'], right_on=['Address', 'Zip']).
    drop(['in_street', 'in_zip'], 1)
)

df_full.to_csv(os.path.join(DATA_DIR, OUT_FILE), index=False)